In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from config import db_password,g_key
from config import resolution,lat_max,lat_min,lng_max,lng_min
import json
from shapely.geometry import Point,shape
import os
from datetime import datetime,timedelta
from functions import Rmax_calc,recovery_calc,month_index

# Read files

In [2]:
# blocks information
file_path = os.path.join('..','..','Resources','blocks_info.csv')
blocks_df = pd.read_csv(file_path)

# Recovery Calculation

In [3]:
# Convert to datetime format
blocks_df['stack_finish'] = pd.to_datetime(blocks_df['stack_finish'])
blocks_df['leach_start'] = pd.to_datetime(blocks_df['leach_start'])
blocks_df['leach_end'] = pd.to_datetime(blocks_df['leach_end'])
blocks_df['leach_days'] = blocks_df['leach_end']-blocks_df['leach_start']
lat_lng_list = blocks_df[['lat','lng']].values.tolist()

In [4]:
block_leach_time = pd.DataFrame(blocks_df[['lift']])
block_leach_time['lat_lng']=blocks_df[['lat','lng']].values.tolist()
block_leach_time.index.name = 'block_id'
block_list = block_leach_time.index.tolist()
block_list[:5]

[0, 1, 2, 3, 4]

In [5]:
project_span = (
    datetime(year=2019,month=1,day=1),
    datetime(year=2031,month=12,day=31)
    )
project_days = (project_span[1] - project_span[0]).days
# Create timeline by day
datelist = pd.date_range(project_span[0], periods=project_days).tolist()
# Create timeline by month
startyear = project_span[0].year
startmonth = project_span[0].month
endyear = project_span[1].year
endmonth = project_span[1].month
monthlist = [datetime(m//12, m%12+1, 1) for m in range(startyear*12+startmonth-1, endyear*12+endmonth)]
# Choose days or months
# rec_oz_df = pd.DataFrame(index=datelist)
rec_oz_df = pd.DataFrame(monthlist,columns = ['months'])
#rec_oz_df['oz']=0
#rec_oz_df['oz'] = rec_oz_df['oz'].astype(float)
rec_oz_df.head()

,months
0,2019-01-01
1,2019-02-01
2,2019-03-01
3,2019-04-01
4,2019-05-01


In [18]:
blocks_df.iloc[0]

lat                              65.012
lng                         -147.351429
lift                                  1
cell                                  1
stack_finish        2019-03-27 00:00:00
leach_start         2019-03-30 00:00:00
leach_end           2019-07-28 00:00:00
leach_days            120 days 00:00:00
tons_per_block              1994.483436
ounces_per_block              16.809887
rmax                           0.706002
Name: 0, dtype: object

In [19]:
month_index(datetime(2019,3,27),project_span)

2

In [21]:
for i in range(0,2):
    print(i)

0
1


In [28]:
# This block of code calculate a specific block's recovery over months.
k=0.1
def block_months_rec(i,ref,k):
    block = blocks_df.iloc[i]
    if i == ref:
        start_date = block['leach_start']
        end_date = block['leach_end']
    else:
        ref_block = blocks_df.iloc[ref]
        start_date = ref_block['leach_start']
        end_date = ref_block['leach_end']
    m_index = month_index(start_date,project_span)
    start_month = start_date.month
    end_month = end_date.month + (end_date.year - start_date.year) * 12
    rmax = block['rmax']
    rcum = sum(rec_oz_df.iloc[0:m_index][i])
    if rcum<1:
        for m in range(start_month,end_month+1):
            if m == start_month:
                if m == 12:
                    days = 31 - start_date.day
                else:
                    days = (datetime(start_date.year,start_date.month+1,1)-start_date).days
            elif m == end_month:
                days = end_date.day
            elif m < 12:
                days = (datetime(start_date.year,m+1,1)-datetime(start_date.year,m,1)).days
            elif m == 12:
                days = 31
            else:
                days = (datetime(start_date.year+1,m-12+1,1)-datetime(start_date.year+1,m-12,1)).days
            # recovery calculation here need to change the k for multiple options.
            r = recovery_calc(rmax,rcum,k,days)
            
            if rcum + r > 1:
                r = 1 - rcum
                rcum = 1
            else:
                rcum += r
            rec_oz_df.at[m_index,i] = r
            m_index += 1
    
        #print(f'Month {m} has {days} days. Recovery is {r}')

In [29]:
for high in block_list:
    this_block = blocks_df.iloc[high]
    rec_oz_df[high]=0.0
    block_months_rec(high,high,k)
    for low in range(0,high):
        if lat_lng_list[high]==lat_lng_list[low]:
            #print(f'{high} {low}')
            block_months_rec(low,high,k)

In [26]:
sum(rec_oz_df[126])

0.6858064043953291

In [27]:
blocks_df.head()

,lat,lng,lift,cell,stack_finish,leach_start,leach_end,leach_days,tons_per_block,ounces_per_block,rmax
block_id,,,,,,,,,,,
0,65.012000,-147.351429,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
1,65.012000,-147.350612,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
2,65.012265,-147.353061,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
3,65.012265,-147.352245,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
4,65.012265,-147.351429,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002


In [8]:
rec_oz_df.head()

,months,0,1,2,3,4,5,6,7,8,...,5139,5140,5141,5142,5143,5144,5145,5146,5147,5148
0,2019-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-03-01,0.009830,0.009830,0.009830,0.009830,0.009830,0.009830,0.009830,0.009830,0.009830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-04-01,0.120277,0.120277,0.120277,0.120277,0.120277,0.120277,0.120277,0.120277,0.120277,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-05-01,0.090009,0.090009,0.090009,0.090009,0.090009,0.090009,0.090009,0.090009,0.090009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def secondary_leach(reference,i):
    secondary_dates=[]
    for b in range(0,i):
        block = blocks_df.iloc[b]
        if block['lat'] == reference['lat'] and block['lng'] == reference['lng']:
            if block['lift'] > reference['lift']:
                secondary_dates.append((block['leach_start'],block['leach_end']))
    reference['secondary_leach'] = secondary_dates

In [12]:
for b in blocks_df.index:
    secondary_leach(blocks_df.iloc[b],b)

<ipython-input-11-c0a178afd031>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reference['secondary_leach'] = secondary_dates


In [20]:
print(blocks_df.iloc[100]['secondary_leach'])

In [12]:
# Create a list of on/off date pairs for each block
reference = blocks_df.iloc[1000]
for block in blocks_df.index:
    if blocks_df.iloc[block]['lat'] == reference['lat'] and blocks_df.iloc[block]['lng'] == reference['lng']:
        if blocks_df.iloc[block]['lift']<reference['lift']:
            print(blocks_df.iloc[block])

lat                           65.017571
lng                         -147.359592
lift                                  1
cell                                  3
stack_finish        2019-05-26 00:00:00
leach_start         2019-05-29 00:00:00
leach_end           2019-09-26 00:00:00
leach_days            120 days 00:00:00
tons_per_block               1976.39033
ounces_per_block              16.621023
rmax                            0.70575
Name: 726, dtype: object
lat                           65.017571
lng                         -147.359592
lift                                  2
cell                                  8
stack_finish        2019-11-02 00:00:00
leach_start         2019-11-05 00:00:00
leach_end           2020-02-05 00:00:00
leach_days             92 days 00:00:00
tons_per_block               165.562914
ounces_per_block               2.490284
rmax                           0.741147
Name: 1206, dtype: object


In [ ]:

# Create a dataframe with months as index, blocks as columns
# Create a dataframe with blocks as index, months as columns

# Create a function that calculates the gold production change for a block for its life span. Apply this function to the block series in the dataframe

In [9]:
def primary_leach(b,k,i):
    r_cum=0
    # want to add this block into the dataframe as a column.
    # this column stores the ounces change through time.
    # the maximum ounces appears when stacking is finished.
    # also need to keep the ounces left to the end of the project
    # maybe create a list specifically for this block and then add this list to the master dataframe.
    # rec_oz_df[str(i)] = 0
    # rec_oz_df[str(i)] = rec_oz_df[str(t)].astype(float)
    # rec_oz_df.at[blocks_df.iloc[i]['stack_finish'],str(i)]=blocks_df.iloc[i]['ounces_per_block']
    
    for n in range(1,b['leach_days'].days+1):
        r_date = b['leach_start'] + timedelta(days = n-1)
        r_day = recovery_calc(b['rmax'],r_cum,k,1)
        oz_day = r_day * b['ounces_per_block']
        rec_oz_df.at[r_date,'oz'] += oz_day
        rec_oz_df.at[r_date,str(i)] = rec_oz_df.at[r_date-timedelta(days = 1),str(i)]-oz_day
        r_cum += r_day

In [12]:
def primary_leach_by_month(k):
    recovery_list = recovery_calc(blocks_df['rmax'],Rcum,k,days)

## Think about create a dataframe store each block's remaining ounces throughout project life span. The ounces should be available at the date when stacking is finished.

In [30]:
for i in range(0,len(blocks_df)):
    primary_leach(blocks_df.iloc[i],0.01,i)
# apply the function on the dataframe instead of looping through rows.
rec_oz_df.describe()

,oz,0,1,2,3,4,5,6,7,8,...,5139,5140,5141,5142,5143,5144,5145,5146,5147,5148
count,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,...,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000
mean,13.758694,16.786409,16.786409,16.786409,16.786409,16.786409,16.786409,16.786409,16.786409,16.786409,...,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511
std,27.894489,0.305535,0.305535,0.305535,0.305535,0.305535,0.305535,0.305535,0.305535,0.305535,...,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912
min,0.000000,11.366826,11.366826,11.366826,11.366826,11.366826,11.366826,11.366826,11.366826,11.366826,...,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575
25%,0.000000,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,...,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869
50%,0.000000,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,...,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869
75%,16.621023,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,...,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869
max,175.521822,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,...,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869


In [25]:
blocks_df.iloc[0]

lat                              65.012
lng                         -147.351429
lift                                  1
cell                                  1
stack_finish        2019-03-27 00:00:00
leach_start         2019-03-30 00:00:00
leach_end           2019-07-28 00:00:00
leach_days            120 days 00:00:00
tons_per_block              1994.483436
ounces_per_block              16.809887
rmax                           0.706002
Name: 0, dtype: object